In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
import stan

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
bpath = '~/source/rethinking/'

In [5]:
df = pd.read_csv(bpath + 'data/foxes.csv', sep=';')

In [6]:
df.head()

,group,avgfood,groupsize,area,weight
0,1,0.37,2,1.09,5.02
1,1,0.37,2,1.09,2.84
2,2,0.53,2,2.05,5.33
3,2,0.53,2,2.05,6.07
4,3,0.49,2,2.12,5.85


In [7]:
scaler = StandardScaler()

In [8]:
X_unscaled = df[['avgfood','groupsize','area','weight']]

In [9]:
scaler.fit(X_unscaled)

StandardScaler()

In [10]:
X_scaled = scaler.transform(X_unscaled)

In [13]:
df_scaled = pd.DataFrame(X_scaled, columns=['avgfood','groupsize','area','weight'])

In [14]:
df_scaled[['group']] = df[['group']]

In [15]:
df_scaled.head()

,avgfood,groupsize,area,weight,group
0,-1.933180,-1.530701,-2.249313,0.415931,1
1,-1.933180,-1.530701,-2.249313,-1.433238,1
2,-1.122886,-1.530701,-1.210738,0.678887,2
3,-1.122886,-1.530701,-1.210738,1.306586,2
4,-1.325459,-1.530701,-1.135008,1.119973,3


In [17]:
model_code = """
data {
  int<lower=0> N;
  vector[N] x;
  vector[N] y;
}
parameters {
  real alpha;
  real beta;
  real sigma;
}
model {
  y ~ normal(alpha + beta * x, sigma);
  alpha ~ normal(0, 0.2);
  beta ~ normal(0, 0.5);
  sigma ~ exponential(1);
}
"""

In [18]:
model_data = {
    "N": len(df_scaled),
    "x": df_scaled['area'].to_numpy(),
    "y": df_scaled['weight'].to_numpy()
}

In [19]:
posterior = stan.build(model_code, data=model_data, random_seed=456)

Building...

Building: 52.9s, done.Messages from stanc:
Warning at '/tmp/httpstan_z0dvjq1k/model_olqav2c4.stan', line 13, column 31 to column 36:
  A normal distribution is given parameter sigma as a scale parameter
  (argument 2), but sigma was not constrained to be strictly positive.
Warning at '/tmp/httpstan_z0dvjq1k/model_olqav2c4.stan', line 16, column 2 to column 7:
  Parameter sigma is given a exponential distribution, which has strictly
  positive support, but sigma was not constrained to be strictly positive.


In [20]:
fit = posterior.sample(num_chains=4, num_samples=1000)

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Rejecting initial value:
    Error evaluating the log probability at the initial value.
  Exception: normal_lpdf: Scale parameter is -1.97417, but must be positive! (in '/tmp/httpstan_95upmng9/model_olqav2c4.stan', line 13, column 2 to column 38)
  Gradient evaluation took 0.000184 seconds
  1000 transitions using 10 leapfrog steps per transition would take 1.84 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: normal_lpdf: Scale parameter is -3638.49, but must be positive! (in '/tmp/httpstan_95upmng9/model_olqav2c4.stan', line 13, column 2 to column 38)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the 

In [60]:
def model_summary(fit, params='all', quantiles=[0.05, 0.95]):
    output_df = pd.DataFrame()

    if params == 'all':
        params = [x for x in fit.param_names]

    for param in params:
        this_df = pd.DataFrame()
        this_df['param'] = [param]
        this_df['mean'] = np.mean(fit[param])
        this_df['std'] = np.mean(fit[param])
        this_df[[str(100*x)+'%' for x in quantiles]] = np.quantile(fit[param], quantiles)

        output_df = pd.concat([output_df, this_df])

    return(output_df)


In [61]:
model_summary(fit)

,param,mean,std,5.0%,95.0%
0,alpha,-0.001534,-0.001534,-0.148443,0.139340
0,beta,0.017751,0.017751,-0.133111,0.175946
0,sigma,1.017206,1.017206,0.907957,1.135846


In [26]:
quantiles = [0.05, 0.25, 0.75, 0.95]

In [29]:
[str(100*x)+'%' for x in quantiles]

['5.0%', '25.0%', '75.0%', '95.0%']

In [22]:
np.mean(fit['alpha'])

-0.0015335424339292156

In [25]:
np.quantile(fit['sigma'],[0.05, 0.95])

array([0.9079568 , 1.13584648])